# Zombie Clusters

Partons d’une invasion de zombies. Nous travaillons pour un organisme de santé qui, pour mesurer l’ampleur de la contamination a besoin de déterminer les foyers de propagation. La propagation part d’un zombie qui contamine ses voisins sur un quadrillage. Il peut les contaminer « à gauche », « à droite », « en haut » et « en bas » (donc pas en diagonale).

Pour donner un exemple, il y a ici deux foyers de propagation (un `0` indique un secteur sain, un `1` indique un secteur contaminé) :

```
0 0 0 0 1
0 0 1 1 1
1 1 0 0 0
1 1 1 0 0
```

Les données sont structurées dans un tableau de chaînes (un élément par ligne, chaque ligne est représentée par une chaîne). Pour l’exemple ci-dessus :

```
val data = Array(
    "00001",
    "00111",
    "11000",
    "11100"
)
```

On partira d’une fonction avec une signature de ce genre :

```
def countClusters(data: Array[String]): Int = ???
```

(la signature est en Scala mais peu importe le langage, en pratique)

----

Représentons les données comme un tableau de chaînes :

In [16]:
data = [
    "00001",
    "00111",
    "11000",
    "11101",
]

J'ai envie de ramener le problème à un problème de graphes : compter le nombre de clusters revient à compter le nombre de composantes fortement connexes dans le graphe.

Un noeud sera représenté par ses coordonnées dans la grille : une paire d'entiers.

Le graphe sera représenté par un dictionnaire, dont les clés sont les noeuds et les valeurs les successeurs de ce noeud (ses voisins directement contaminés).

In [17]:
from typing import Dict, List, Set, Tuple


Node = Tuple[int, int]

Graph = Dict[Node, Set[Node]]

On peut commencer par construire l'ensemble des noeuds à partir de notre liste de chaînes :

In [18]:
def nodes(lines: List[str]) -> Set[Node]:
    return {
        (x, y)
        for y, line in enumerate(lines)
        for x, char in enumerate(line)
        if char == "1"
    }


nodes(data)

{(0, 2),
 (0, 3),
 (1, 2),
 (1, 3),
 (2, 1),
 (2, 3),
 (3, 1),
 (4, 0),
 (4, 1),
 (4, 3)}

On peut ensuite construire le graphe lui-même :

In [19]:
from collections import defaultdict


def make_graph(lines: List[str]) -> Graph:
    first_col, last_col = 0, array_width(lines) - 1
    first_line, last_line = 0, array_height(lines) - 1

    edges = defaultdict(set)

    for (x, y) in nodes(lines):

        # add key for node
        edges.setdefault((x, y), set())

        # add edges to/from node above?
        if y > first_line and lines[y - 1][x] == "1":
            edges[(x, y)].add((x, y - 1))
            edges[(x, y - 1)].add((x, y))

        # add edges to/from node below?
        if y < last_line and lines[y + 1][x] == "1":
            edges[(x, y)].add((x, y + 1))
            edges[(x, y + 1)].add((x, y))

        # add edges to/from node left?
        if x > first_col and lines[y][x - 1] == "1":
            edges[(x, y)].add((x - 1, y))
            edges[(x - 1, y)].add((x, y))

        # add edges to/from node right?
        if x < last_col and lines[y][x + 1] == "1":
            edges[(x, y)].add((x + 1, y))
            edges[(x + 1, y)].add((x, y))

    return edges


def array_width(lines: List[str]) -> int:
    if lines:
        return len(lines[0])
    return 0


def array_height(lines: List[str]) -> int:
    return len(lines)


make_graph(data)

defaultdict(set,
            {(1, 2): {(0, 2), (1, 3)},
             (1, 3): {(0, 3), (1, 2), (2, 3)},
             (0, 2): {(0, 3), (1, 2)},
             (0, 3): {(0, 2), (1, 3)},
             (2, 3): {(1, 3)},
             (3, 1): {(2, 1), (4, 1)},
             (2, 1): {(3, 1)},
             (4, 1): {(3, 1), (4, 0)},
             (4, 3): set(),
             (4, 0): {(4, 1)}})

In [20]:
def transitive_closure(graph: Graph) -> Graph:
    """Algorithme de Roy-Warshall"""
    g = graph.copy()
    for x in graph:
        for y in graph:
            if x in graph[y]:
                for z in graph:
                    if z in graph[x]:
                        g[y].add(z)
    return g



transitive_closure(make_graph(data))

defaultdict(set,
            {(1, 2): {(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)},
             (1, 3): {(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)},
             (0, 2): {(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)},
             (0, 3): {(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)},
             (2, 3): {(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)},
             (3, 1): {(2, 1), (3, 1), (4, 0), (4, 1)},
             (2, 1): {(2, 1), (3, 1), (4, 0), (4, 1)},
             (4, 1): {(2, 1), (3, 1), (4, 0), (4, 1)},
             (4, 3): set(),
             (4, 0): {(2, 1), (3, 1), (4, 0), (4, 1)}})

In [21]:
def connected_components(graph: Graph) -> List[Set[Node]]:
    return set(
        frozenset({node} | successors)
        for node, successors in transitive_closure(graph).items()
    )


graph = make_graph(data)
connected_components(graph)

{frozenset({(4, 3)}),
 frozenset({(2, 1), (3, 1), (4, 0), (4, 1)}),
 frozenset({(0, 2), (0, 3), (1, 2), (1, 3), (2, 3)})}

Pour connaître le nombre de foyers de contamination de zombies, on n'a maintenant qu'à compter le nombre de composantes connexes de notre graphe :

In [22]:
def count_clusters(data: List[str]) -> int:
    return len(connected_components(make_graph(data)))


print(count_clusters(data))

3


Quelques tests...

In [23]:
assert count_clusters([]) == 0

In [24]:
assert count_clusters([""]) == 0

In [25]:
assert count_clusters(["0"]) == 0

In [26]:
assert count_clusters(["1"]) == 1

In [27]:
assert count_clusters(["101"]) == 2

In [28]:
assert count_clusters(["1010011"]) == 3

In [29]:
assert count_clusters([
    "101",
    "010",
]) == 3

In [30]:
assert count_clusters([
    "111",
    "111",
    "000",
    "111",
]) == 2